<a href="https://colab.research.google.com/github/Priyo-prog/Machine-Learning/blob/main/Decision%20Tree/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Classification

* In this lesson we are going to use **scikit-learn** and **Cost-Complexity Pruning** to build the Classification tree.
* **Classification Trees** are exceptionally useful machine learning method when you need to know how the decisions are being made.


## Import libraries and modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

## Import the data

* We are goin to load the dataset from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) . Specifically we are going to use [Heart Disease Dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data). This dataset will allow us to predict if someone has heart disease based ojn their age, sex, blood pressure and variety of other metrics.

* We need to replace the column numbers with the following column names:
** age 
** sex 
** cp, chest pain
** restbp, resting blood pressure (in mm Hg)
** fbs, fasting blood sugar
** restecg, resting electrocardiographic results
** thalac, maximum heart rate achieved
** exang, exercise induced angina
** oldpeak, ST depression induced by exercise relative to rest 
** slope, the slope of the peak exercise ST segment
** ca, number of major vessels (0-3) colored by fluroscopy
** thal, this is short of thalium heart scan
** hd, diagnosis of heart disease, the predicted attribute

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [3]:
## Change the column numbers to column names
df.columns = ['age',
              'sex',
              'cp',
              'restbp',
              'chol',
              'fbs',
              'restecg',
              'thalach',
              'exang',
              'oldpeak',
              'slope',
              'ca',
              'thal',
              'hd']
df.head()              

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


## Missing Data Part 1 : Identifying missing data

This part will involve the process of identifying and dealing with **Missing Data**.

**Missing Data** is simply a blank space, or a surrogate value like **NA**, that indicates that we failed to collect data for one of the features.

There are two main ways to deal with missing data:
1. We can remove the rows that contain missing data from the dataset. This is relatively easy to do, but it wastes all other values that we collected.

2. We can **impute** the values that are missing. In this context **impute** is just a fancy way of saying "we can make an educated guess about what the value should be".


In [4]:
## dtypes tell us the "data type" for each column
df.dtypes

age        float64
sex        float64
cp         float64
restbp     float64
chol       float64
fbs        float64
restecg    float64
thalach    float64
exang      float64
oldpeak    float64
slope      float64
ca          object
thal        object
hd           int64
dtype: object

We can see that they are almost all float64, however columns like **ca** and **thal** have object type and one column , **hd** has int64.

Object datatypes are used when there are mixture of things, like a mixture of numbers and letters. 

In [5]:
# print out unique values in the column called 'ca'
df['ca'].unique()

array(['0.0', '3.0', '2.0', '1.0', '?'], dtype=object)

## Missing Data Part 2 : Dealing with missing data

Since scikit-learn classification trees do not support datasets with missing values, we need to figure out what to do these question marks. We can either delete these patients from the training dataset, or impute values for the missing data.

In [6]:
## print the number of rows that contain missing values
##
## loc[], short for "location", let's us specify which rows we want..
## and so we say we want any row with '?' in column 'ca'
## OR
## any row with '?' in column 'thal'
##
## 'len' short for "length", prints out the number of rows
len(df.loc[(df['ca']=='?')
            |
           (df['thal']=='?')])

6

Since only 6 rows have missing values, let's look at them

In [7]:
df.loc[(df['ca']=='?')
            |
           (df['thal']=='?')]

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,hd
87,53.0,0.0,3.0,128.0,216.0,0.0,2.0,115.0,0.0,0.0,1.0,0.0,?,0
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,?,3.0,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,?,7.0,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,?,7.0,0
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


Let's count the number of rows in full dataset

In [8]:
len(df)

303

So 6 of **303** rows, or **2%** contain missing values. Since **303 - 6 = 297**, and **297** is plenty of data to build classification tree, we will remove the rows with missing values, rather than try to impute their values.

In [9]:
## use loc[] to select all the rows that do not contain missing values 
## annd save them in a new dataframe called "df_no_missing"
df_no_missing = df.loc[(df['ca'] != '?')
                       & 
                       (df['thal'] != '?')]

In [10]:
len(df_no_missing)

297

In [11]:
df_no_missing['ca'].unique(), df_no_missing['thal'].unique()

(array(['0.0', '3.0', '2.0', '1.0'], dtype=object),
 array(['6.0', '3.0', '7.0'], dtype=object))

## Format Data Part 1 : Split the data into features and labels

In [12]:
## make a new copy of the columns used to make predictions
X = df_no_missing.drop('hd', axis=1).copy() # alternatively df_no_missing[:, :-1]
X.head()

,age,sex,cp,restbp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0


In [13]:
## Make a new column of the data we want to predict
y = df_no_missing['hd'].copy()
y.head()

0    0
1    2
2    1
3    0
4    0
Name: hd, dtype: int64